<a href="https://colab.research.google.com/github/DhruvaBansal00/CV-Spring2020-Project/blob/main/baseline_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import tensorflow_hub as hub

import tensorflow_datasets as tfds


In [ ]:
# Import dataset data from tfds

NUM_CLASSES = 10
dataset_name = 'cifar'+str(NUM_CLASSES)
ds_train, info = tfds.load(name=dataset_name, split='train', with_info=True)
ds_test = tfds.load(name=dataset_name, split='test')
train_examples = info.splits['train'].num_examples

In [ ]:
# Split into train and test sets
# We have checked that the classes are reasonably balanced.
# train_split = 0.98
# num_train = int(train_split * num_examples)
# ds_train = ds_train.take(num_train)
DATASET_NUM_TRAIN_EXAMPLES = train_examples

In [ ]:
# Load model from TFHub into KerasLayer
efficient_net_url = "https://tfhub.dev/google/efficientnet/b3/feature-vector/1"
efficientNetKeras = hub.KerasLayer(efficient_net_url)

In [ ]:
class EfficientNetModel(tf.keras.Model):
  """Efficient Net with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = EfficientNetModel(num_classes=NUM_CLASSES, module=efficientNetKeras)

In [ ]:

# Preprocessing helper functions
RESIZE_TO = 160
CROP_TO = 128
# Create data pipelines for training and testing:
BATCH_SIZE = 512
SCHEDULE_LENGTH = 20
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE

STEPS_PER_EPOCH = 1

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (ds_train
                  .shuffle(200000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

pipeline_test = (ds_test.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))

In [ ]:
SCHEDULE_BOUNDARIES = [3000, 6000, 9000]
lr = 0.003 * BATCH_SIZE / 512 
# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH), 
    validation_data=pipeline_test  
)

print(history)

########################################################################################

Pipeline Generating plots:

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from numpy.random import seed
seed(42)
tf.random.set_seed(42)

In [8]:
#Model Definition
class EfficientNetModel(tf.keras.Model):
  """Efficient Net with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
    self.softmax_layer = tf.keras.layers.Softmax()
  
  def call(self, images):
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

In [9]:
#Dataset Definition

classes = 10
dataset_name = 'cifar'+str(classes)
train_dataset, info = tfds.load(name=dataset_name, split='train', with_info=True)
test_dataset = tfds.load(name=dataset_name, split='test')
num_train_examples = info.splits['train'].num_examples

In [10]:
#Pre-processing methods
RESIZE_TO = 300
CROP_TO = 300

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  # features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

In [ ]:
#Setting Up Training Parameters. Note that Epochs = SCHEDULE_LENGTH/STEPS_PER_EPOCH.
#Steps per epoch = number of times you want to repeat the training set before evaluating on the test set
#Schedule Length = total number of times you want to repeat the trainig set. 
BATCH_SIZE = 128
SCHEDULE_LENGTH = 100
STEPS_PER_EPOCH = 100
SCHEDULE_BOUNDARIES = [20, 50, 80]
learning_rate = 0.0003 

train_data_sizes = [0.1, 1]
final_train_accuracies = []
final_test_accuracies = []

for curr_size in train_data_sizes:

  #Model Instantiation
  efficientNetKeras = hub.KerasLayer("https://tfhub.dev/google/efficientnet/b7/feature-vector/1")
  efficientNet = EfficientNetModel(num_classes=classes, module=efficientNetKeras)

  curr_train_examples_num = int(curr_size * num_train_examples)
  curr_train_dataset = train_dataset.take(curr_train_examples_num)

  print(curr_train_examples_num)
  print(len(curr_train_dataset))

  pipeline_train = (curr_train_dataset
                  .shuffle(200000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / curr_train_examples_num * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

  pipeline_test = (test_dataset.map(preprocess_test, num_parallel_calls=1)
                    .map(cast_to_tuple)  # for keras model.fit
                    .batch(BATCH_SIZE)
                    .prefetch(2))
  
  
  # Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
  lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                    values=[learning_rate, learning_rate*0.1, learning_rate*0.001, learning_rate*0.0001])
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  efficientNet.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

  # Fine-tune model
  history = efficientNet.fit(
      pipeline_train,
      batch_size=BATCH_SIZE,
      steps_per_epoch=STEPS_PER_EPOCH,
      epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH), 
      validation_data=pipeline_test  
  )








5000
5000
 56/100 [===============>..............] - ETA: 1:49 - loss: 1.6896 - accuracy: 0.8521